In [30]:
import pandas as pd
pd.set_option('display.max_columns', 55)

In [31]:
weekly = pd.read_parquet('data/weekly.parquet')
weekly = weekly.loc[weekly.index != 3821]
weekly['headshot_url'] = weekly['headshot_url'].fillna('no_url')

In [ ]:
depth = pd.read_csv('data/depth.csv', index_col=0).sort_values(by='depth_team')
off = depth.loc[depth.formation=='Offense'].copy()
off

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth
25616,2010,ARI,1.0,REG,1,Brown,Levi,Levi,Offense,00-0025392,67.0,T,BRO548586,LT,Levi Brown,0.0
262548,2017,DAL,8.0,REG,1,Martin,Zachary,Zack,Offense,00-0031402,70.0,G,MAR771661,RG,Zack Martin,0.0
262547,2017,DAL,8.0,REG,1,Smith,Tyron,Tyron,Offense,00-0027947,77.0,T,SMI766760,LT,Tyron Smith,0.0
489647,2023,KC,4.0,REG,1,Kelce,Travis,Travis,Offense,00-0030506,87.0,TE,KEL012458,TE,Travis Kelce,0.0
489651,2023,KC,4.0,REG,1,Toney,Kadarius,Kadarius,Offense,00-0036913,19.0,WR,TON593115,WR,Kadarius Toney,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299424,2018,DAL,15.0,REG,3,Olawale,Jamize,Jamize,Offense,00-0029492,49.0,FB,OLA706032,RB,Jamize Olawale,0.0
299427,2018,DAL,16.0,REG,3,Martin,Zachary,Zack,Offense,00-0031402,70.0,G,MAR771661,RG,Zack Martin,1.0
299430,2018,DAL,16.0,REG,3,Lenoir,Lance,Lance,Offense,00-0033992,34.0,WR,LEN392908,WR,Lance Lenoir,6.0
299407,2018,DAL,15.0,REG,3,Gathers,Rico,Rico,Offense,00-0033054,82.0,TE,GAT444006,TE,Rico Gathers,3.0


In [33]:
context_cols = ['player_id', 'player_display_name','position',
       'position_group','headshot_url'
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', ]
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr']
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'full_name']

In [34]:
qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "interceptions",
    "sacks",
    "sack_yards",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "fantasy_points_ppr"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
]


In [35]:
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))

In [61]:
weekly.loc[
    (weekly.player_id=='00-0023108')
    & (weekly.season==2011)
][["player_id", "recent_team", "season", "week", "position"]]

,player_id,recent_team,season,week,position
1050,00-0023108,LA,2011,11,TE
1051,00-0023108,LA,2011,12,TE


In [ ]:
dork_off = off.loc[off.gsis_id=='00-0023108'].copy()
dork_week = weekly.loc[weekly.player_id=='00-0023108'].copy()
dork_temp = dork_off.merge(
    dork_week,
    how="outer",
    left_on=["gsis_id", "team_name", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
).sort_values(by="week").reset_index(drop=True)
week_merge_cols = ["player_id", "recent_team", "season", "week", "position"]
off_merge_cols = ["player_id", "recent_team", "season", "week", "position"]
dork_temp

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,player_id,player_name,player_display_name,position_group,headshot_url,recent_team,season_type,opponent_team,completions,attempts,passing_yards,...,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,2010,ARI,1.0,REG,2.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,1.0,00-0023108,None,Stephen Spach,TE,no_url,ARI,REG,LA,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.564644,0.0,NaN,0.025000,0.000000,0.037500,0.0,0.0,1.0
1,2011,LA,1.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,ARI,2.0,REG,2.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011,LA,2.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011,LA,3.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2012,JAX,3.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010,ARI,3.0,REG,2.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2011,LA,4.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010,ARI,4.0,REG,2.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2011,LA,5.0,REG,3.0,Spach,Stephen,Stephen,Offense,00-0023108,83.0,TE,SPA047461,TE,Stephen Spach,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#week_small = weekly.loc[(weekly.player_id=='00-0039746') & (weekly.season==2024)]
temp = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "team_name", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
).sort_values(by="week").reset_index(drop=True)
temp['headshot_url'] = temp['headshot_url'].fillna('no_url')
#temp[["player_id", "recent_team", "season", "week", "position"]] = temp[["player_id", "recent_team", "season", "week", "position"]].fillna(temp[["gsis_id", "club_code", "season", "week", "position"]])

In [ ]:
off.loc[(off.gsis_id == '00-0027061')
        & (off.season==2015)].sort_values(by='week').merge(
            weekly.loc[(weekly.player_id == '00-0027061')
        & (weekly.season==2015)][["player_id", "recent_team", "season", "week", "position"]],
            how="outer",
    left_on=["gsis_id", "team_name", "season", "week", "position"],
    right_on=["player_id", "recent_team", "season", "week", "position"],
        )

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,player_id,recent_team
0,2015,LA,1.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
1,2015,LA,2.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
2,2015,LA,3.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,0.0,00-0027061,LA
3,2015,LA,4.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
4,2015,LA,5.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
5,2015,LA,6.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,NaN,NaN
6,2015,LA,7.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
7,2015,LA,8.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
8,2015,LA,9.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,1.0,00-0027061,LA
9,2015,LA,10.0,REG,1,Cook,Jared,Jared,Offense,00-0027061,87.0,TE,COO152641,TE,Jared Cook,0.0,00-0027061,LA


In [ ]:
temp[context_cols] = temp.groupby(depth_cols)[context_cols].bfill()
temp[context_cols] = temp.groupby(depth_cols)[context_cols].ffill()
temp[stat_cols] = temp[stat_cols].fillna(0)

In [42]:
weekly.loc[
    weekly.player_id=='00-0023108'
][context_cols + ['week', 'season']].sort_values(by='season')

,player_id,player_display_name,position,position_group,headshot_url,week,season
1502,00-0023108,Stephen Spach,TE,TE,no_url,1,2010
1503,00-0023108,Stephen Spach,TE,TE,no_url,5,2010
1504,00-0023108,Stephen Spach,TE,TE,no_url,7,2010
1505,00-0023108,Stephen Spach,TE,TE,no_url,8,2010
1506,00-0023108,Stephen Spach,TE,TE,no_url,9,2010
1507,00-0023108,Stephen Spach,TE,TE,no_url,10,2010
1508,00-0023108,Stephen Spach,TE,TE,no_url,12,2010
1509,00-0023108,Stephen Spach,TE,TE,no_url,13,2010
1510,00-0023108,Stephen Spach,TE,TE,no_url,14,2010
1511,00-0023108,Stephen Spach,TE,TE,no_url,15,2010


In [43]:
temp.loc[
    temp.gsis_id=='00-0023108'
][context_cols + ['week', 'season']].sort_values(by='season')

,player_id,player_display_name,position,position_group,headshot_url,week,season
206783,00-0023108,Stephen Spach,TE,TE,no_url,18.0,2010
121114,00-0023108,Stephen Spach,TE,TE,no_url,10.0,2010
151053,00-0023108,Stephen Spach,TE,TE,no_url,13.0,2010
110659,00-0023108,Stephen Spach,TE,TE,no_url,9.0,2010
90707,00-0023108,Stephen Spach,TE,TE,no_url,8.0,2010
169018,00-0023108,Stephen Spach,TE,TE,no_url,14.0,2010
85097,00-0023108,Stephen Spach,TE,TE,no_url,7.0,2010
63831,00-0023108,Stephen Spach,TE,TE,no_url,6.0,2010
145628,00-0023108,Stephen Spach,TE,TE,no_url,12.0,2010
180002,00-0023108,Stephen Spach,TE,TE,no_url,15.0,2010


In [44]:
player_fantasy = temp.copy(deep=True).sort_values(by=['season','week']) #[context_cols + game_context + rush_stat_cols]
#player_fantasy.loc[player_fantasy.gsis_id=="00-0039746"]

In [45]:
roll_columns = [x + '_roll' for x in stats_to_eng]
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=6, min_periods=0).mean())
#player_fantasy[['carries_median']] = player_fantasy.groupby(context_cols)[['carries']].transform(lambda x: x.rolling(6, min_periods=0).median())
#player_fantasy.loc[player_fantasy.gsis_id=="00-0039746"][['carries','carries_median']]

In [47]:

player_fantasy[[x for x in player_fantasy.columns if x not in stat_cols]].drop(['recent_team','opponent_team'],axis=1).to_parquet('data/agg/player_weekly_agg.parquet')

In [48]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['team_name','season','week','full_name'])

In [49]:
a =weekly.loc[(weekly.targets > 0) & (weekly.position.isin(["RB", "WR", "TE"]))][
    ["player_display_name", "targets", "week", "season", "position", "recent_team"]
].groupby(["week", "season", "recent_team", "position"]).count().reset_index().groupby(
    ["position"]
)["targets"].value_counts().reset_index()


In [50]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [51]:
cols_to_agg =  (['gsis_id','full_name'] + roll_columns)

team_player_stats = stack_players(agg.loc[(agg.position.isin(['RB','WR'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], cols_to_agg)
agg_roll_cols = [x for x in team_player_stats.columns if '_roll' in x]
team_player_stats[agg_roll_cols] = team_player_stats[agg_roll_cols].fillna(0).astype(float)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_16680\3295183237.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  team_player_stats[agg_roll_cols] = team_player_stats[agg_roll_cols].fillna(0).astype(float)


In [52]:
agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())][['team_name','season','week', 'game_type', 'dense_depth']]

,team_name,season,week,game_type,dense_depth
6335,ARI,2010,1.0,REG,0.0
1223,ARI,2010,1.0,REG,2.0
6957,ARI,2010,1.0,REG,1.0
23866,ARI,2010,2.0,REG,0.0
20220,ARI,2010,2.0,REG,2.0
...,...,...,...,...,...
129773,WAS,2024,11.0,REG,2.0
123242,WAS,2024,11.0,REG,1.0
135784,WAS,2024,12.0,REG,0.0
139658,WAS,2024,12.0,REG,2.0


In [53]:
qb_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in qb_stats_eng] 
receiver_roll_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = ['gsis_id','full_name'] + [x + '_roll' for x in rush_stats_eng]

team_qb_stats = stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], qb_roll_stat_cols)
wr_temp = stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
te_temp = stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
rb_temp =stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],2,  ['team_name','season','week', 'game_type'], receiver_roll_stat_cols)
team_receiver_stats = wr_temp.merge(te_temp, on=['team_name','season','week', 'game_type'])
team_receiver_stats = team_receiver_stats.merge(rb_temp, on=['team_name','season','week', 'game_type'])
team_rb_stats = stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],3,  ['team_name','season','week', 'game_type'], rush_stat_cols)

In [54]:
team_receiver_stats.columns

Index(['team_name', 'season', 'week', 'game_type', 'gsis_id_WR_4',
       'gsis_id_WR_2', 'gsis_id_WR_0', 'gsis_id_WR_3', 'gsis_id_WR_1',
       'full_name_WR_4',
       ...
       'receiving_yards_after_catch_roll_RB_1',
       'receiving_yards_after_catch_roll_RB_0',
       'receiving_first_downs_roll_RB_1', 'receiving_first_downs_roll_RB_0',
       'target_share_roll_RB_1', 'target_share_roll_RB_0',
       'air_yards_share_roll_RB_1', 'air_yards_share_roll_RB_0',
       'fantasy_points_ppr_roll_RB_1', 'fantasy_points_ppr_roll_RB_0'],
      dtype='object', length=124)

In [55]:
team_qb_stats.to_csv('data/team_qb_stats.csv')
team_receiver_stats.to_csv('data/team_receiver_stats.csv')
team_rb_stats.to_csv('data/team_rb_stats.csv')

In [ ]:
agg.loc[
    (agg.team_name=='LA')
    & (agg.season==2024)
    & (agg.week==12)
    & (agg.position=='WR')
]

,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,player_id,player_name,player_display_name,position_group,headshot_url,recent_team,season_type,opponent_team,completions,attempts,passing_yards,...,fantasy_points_ppr,receptions_roll,rushing_tds_roll,passing_2pt_conversions_roll,rushing_first_downs_roll,carries_roll,target_share_roll,air_yards_share_roll,receiving_air_yards_roll,interceptions_roll,passing_yards_roll,passing_first_downs_roll,rushing_yards_roll,fantasy_points_roll,passing_tds_roll,sack_yards_roll,passing_yards_after_catch_roll,receiving_tds_roll,completions_roll,passing_air_yards_roll,receiving_first_downs_roll,attempts_roll,receiving_yards_roll,fantasy_points_ppr_roll,receiving_yards_after_catch_roll,sacks_roll,targets_roll
137719,2024,LA,12.0,REG,1.0,Kupp,Cooper,Cooper,Offense,00-0033908,10.0,WR,KUP534597,WR,Cooper Kupp,2.0,00-0033908,NaN,Cooper Kupp,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136516,2024,LA,12.0,REG,1.0,Robinson,Demarcus,Demarcus,Offense,00-0032775,15.0,WR,ROB399065,WR,Demarcus Robinson,1.0,00-0032775,NaN,Demarcus Robinson,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137060,2024,LA,12.0,REG,2.0,Whittington,Jordan,Jordan,Offense,00-0039751,88.0,WR,WHI807684,WR,Jordan Whittington,3.0,00-0039751,NaN,Jordan Whittington,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135418,2024,LA,12.0,REG,1.0,Nacua,Puka,Puka,Offense,00-0039075,17.0,WR,NAC559347,WR,Puka Nacua,0.0,00-0039075,NaN,Puka Nacua,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
137475,2024,LA,12.0,REG,2.0,Atwell,Chatarius,Tutu,Offense,00-0036849,5.0,WR,ATW460468,WR,Tutu Atwell,4.0,00-0036849,NaN,Tutu Atwell,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139438,2024,LA,12.0,REG,2.0,Johnson,Tyler,Tyler,Offense,00-0036427,18.0,WR,JOH711094,WR,Tyler Johnson,5.0,00-0036427,NaN,Tyler Johnson,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139726,2024,LA,12.0,REG,3.0,Smith,Xavier,Xavier,Offense,00-0038359,19.0,WR,SMI826503,WR,Xavier Smith,6.0,00-0038359,NaN,Xavier Smith,WR,no_url,NaN,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
depth.loc[
    (depth.team_name=='LA')
    & (depth.season==2024)
    & (depth.week==12)
    & (depth.position=='WR')
    & (depth.formation=='Offense')
]

,season,club_code,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth
523482,2024,LA,12.0,REG,1,Nacua,Puka,Puka,Offense,00-0039075,17.0,WR,NAC559347,WR,Puka Nacua,0.0
523454,2024,LA,12.0,REG,1,Robinson,Demarcus,Demarcus,Offense,00-0032775,15.0,WR,ROB399065,WR,Demarcus Robinson,1.0
523450,2024,LA,12.0,REG,1,Kupp,Cooper,Cooper,Offense,00-0033908,10.0,WR,KUP534597,WR,Cooper Kupp,2.0
523448,2024,LA,12.0,REG,2,Whittington,Jordan,Jordan,Offense,00-0039751,88.0,WR,WHI807684,WR,Jordan Whittington,3.0
523467,2024,LA,12.0,REG,2,Atwell,Chatarius,Tutu,Offense,00-0036849,5.0,WR,ATW460468,WR,Tutu Atwell,4.0
523496,2024,LA,12.0,REG,2,Johnson,Tyler,Tyler,Offense,00-0036427,18.0,WR,JOH711094,WR,Tyler Johnson,5.0
523463,2024,LA,12.0,REG,3,Smith,Xavier,Xavier,Offense,00-0038359,19.0,WR,SMI826503,WR,Xavier Smith,6.0


In [58]:
receiver_roll_stat_cols

['gsis_id',
 'full_name',
 'receptions_roll',
 'targets_roll',
 'receiving_yards_roll',
 'receiving_tds_roll',
 'receiving_air_yards_roll',
 'receiving_yards_after_catch_roll',
 'receiving_first_downs_roll',
 'target_share_roll',
 'air_yards_share_roll',
 'fantasy_points_ppr_roll']

In [60]:
agg.loc[agg.player_id=='00-0036322']

,season,team_name,week,game_type,depth_team,last_name,first_name,football_name,formation,gsis_id,jersey_number,position,elias_id,depth_position,full_name,dense_depth,player_id,player_name,player_display_name,position_group,headshot_url,recent_team,season_type,opponent_team,completions,attempts,passing_yards,...,fantasy_points_ppr,receptions_roll,rushing_tds_roll,passing_2pt_conversions_roll,rushing_first_downs_roll,carries_roll,target_share_roll,air_yards_share_roll,receiving_air_yards_roll,interceptions_roll,passing_yards_roll,passing_first_downs_roll,rushing_yards_roll,fantasy_points_roll,passing_tds_roll,sack_yards_roll,passing_yards_after_catch_roll,receiving_tds_roll,completions_roll,passing_air_yards_roll,receiving_first_downs_roll,attempts_roll,receiving_yards_roll,fantasy_points_ppr_roll,receiving_yards_after_catch_roll,sacks_roll,targets_roll
4615,2020,MIN,1.0,REG,2.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,2.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/private/f_aut...,MIN,REG,GB,0.0,0.0,0.0,...,4.60,2.000000,0.0,0.0,0.0,0.000000,0.120000,0.037453,10.000000,0.0,0.000000,0.000000,0.000000,2.600000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,26.000000,4.600000,20.000000,0.0,3.000000
15503,2020,MIN,2.0,REG,2.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,3.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/private/f_aut...,MIN,REG,IND,0.0,0.0,0.0,...,7.40,2.583333,0.0,0.0,0.0,0.000000,0.126087,0.110618,29.250000,0.0,0.000000,0.000000,0.000000,3.650000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.583333,0.000000,36.500000,6.233333,8.916667,0.0,3.000000
26956,2020,MIN,3.0,REG,2.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,3.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/private/f_aut...,MIN,REG,TEN,0.0,0.0,0.0,...,30.50,4.568807,0.0,0.0,0.0,0.000000,0.231241,0.316630,73.642202,0.0,0.000000,0.000000,0.000000,12.573395,0.0,0.0,0.000000,0.449541,0.000000,0.000000,4.018349,0.000000,98.761468,17.142202,40.422018,0.0,5.697248
44839,2020,MIN,4.0,REG,2.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,2.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/private/f_aut...,MIN,REG,HOU,0.0,0.0,0.0,...,14.30,4.349099,0.0,0.0,0.0,0.000000,0.233888,0.316159,79.188063,0.0,0.000000,0.000000,0.000000,11.695270,0.0,0.0,0.000000,0.275901,0.000000,0.000000,4.011261,0.000000,100.398649,16.044369,35.237613,0.0,5.427928
57149,2020,MIN,5.0,REG,2.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,3.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/private/f_aut...,MIN,REG,SEA,0.0,0.0,0.0,...,5.30,3.875603,0.0,0.0,0.0,0.000000,0.197981,0.275127,69.996784,0.0,0.000000,0.000000,0.000000,8.397793,0.0,0.0,0.000000,0.179067,0.000000,0.000000,3.305365,0.000000,73.233884,12.273396,22.870194,0.0,5.277737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95429,2024,MIN,8.0,REG,1.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,1.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/upload/f_auto...,MIN,REG,LA,0.0,0.0,0.0,...,19.50,6.655560,0.0,0.0,0.0,0.225471,0.361853,0.427870,110.855592,0.0,0.000000,0.000000,0.676412,12.816762,0.0,0.0,0.000000,0.523290,0.000000,0.000000,4.594426,0.000000,96.093789,19.472321,22.315876,0.0,9.139425
99562,2024,MIN,9.0,REG,1.0,Jefferson,Justin,Justin,Offense,00-0036322,18.0,WR,JEF269287,WR,Justin Jefferson,1.0,00-0036322,J.Jefferson,Justin Jefferson,WR,https://static.www.nfl.com/image/upload/f_auto...,MIN,REG,IND,1.0,1.0,22.0,...,21.58,6.761124,0.0,0.0,0.0,0.1